In [1]:
import sys
import csv
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import pickle 
import pprint
import requests
from datetime import date, datetime
from random import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import os
import datetime
import shutil
import glob

In [2]:
pp = pprint.PrettyPrinter(indent=4)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("disable-gpu")

## camping site 정보 수집 (부대시설 정보 추가해야함)

In [761]:
chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chrome_path, options=options)

url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?pageUnit=2500&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1'

def get_siteinfo(url):
    driver.get(url) 

    time.sleep(10)

    source = driver.page_source 
    data = bs(source, 'html.parser')
    cont = data.findAll("div",{"class":"camp_cont"})

    cols = ['name','url','addr']
    df = pd.DataFrame(columns=cols)
    prefix = 'https://www.gocamping.or.kr'
    for i in range(len(cont)):
        name = cont[i].h2.a.get_text().split("]")[1]
        url_t = cont[i].h2.a['href'].split("&")[0]
        url = ''.join([prefix,url_t])
        addr = cont[i].ul(class_="addr")[0].get_text()

        info = [name, url, addr]
        dict1 = dict(zip(cols, info))
        df = df.append(dict1, ignore_index=True)

    df['name'] = df.name.str.strip()
#     df['hashtag'] = df.name.str.replace(' ','')
#     df['hashtag'] = df.hashtag.str.replace('(주)','')
    return df

## Get xy coordinate directly from embeded navermap

In [641]:
def get_sitexy(bsobj):
    cont = bsobj.findAll("div", {"class":"map_list"})[0]
    ul = cont.ul.select("li[onclick]")
    cols = ['name','addr', 'call_num','coord_x','coord_y']
    df = pd.DataFrame(columns=cols)

    for i in range(len(ul)):
        camp = ul[i]
        xy = camp['onclick']
        x = re.findall('[0-9]*[0-9]\.[0/-9]*[0-9]', xy)[1]
        y = re.findall('[0-9]*[0-9]\.[0/-9]*[0-9]', xy)[0]
        name = camp.h2.text.split(']')[1].strip()
        addr = camp.find("li", {"class":"addr"}).get_text()
        call_num = camp.find("li", {"class":"call_num"}).get_text() if camp.find("li", {"class":"call_num"}) else ""
        val_list = [name, addr, call_num, x, y]
        result = dict(zip(cols, val_list))
        df = df.append(result, ignore_index=True)
    print(df.head(2))
    return df

In [719]:
cols = ['name','addr', 'call_num','coord_x','coord_y']
df_fn = pd.DataFrame(columns=cols)
limit_ = 1000

chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chrome_path, options=options)

for i in range(1,4):
#     print(i)
    url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit={limit}&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex={page}'.format(limit=limit_, page=i)
    print(url)
    driver.get(url) 
    time.sleep(3)
    source = driver.page_source 
    data = bs(source, 'html.parser')
    
    df_rslt = get_sitexy(data)
#     print(df_rslt.head())
    df_fn = df_fn.append(df_rslt)
    

https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1000&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1
       name                       addr       call_num      coord_x     coord_y
0     트리캠핑장  인천광역시 옹진군 선재로306번길 27-55                  126.5149936  37.2568236
1  마노아워터캠핑장          경남 합천군 봉산면 압곡3길 9  010-8667-6051  128.0578454  35.6424478
https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1000&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=2
         name                      addr       call_num      coord_x  \
0  평화의댐 오토캠핑장  강원 화천군 화천읍 비수구미길 145-145  070-7727-1290  127.8501067   
1      펫 트리파크     충북 충주시 앙성면 둔터로 625-31                 127.7103409   

      coord_y  
0  38.2065288  
1  37.1029014  
https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1000&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=3
          name                      addr       call_num      coord_x  \
0  제이드가든 오토캠핑장        강원도 춘천시 

In [720]:
df = df_fn.reset_index(drop=True)

In [791]:
def name_for_tag(x):
    if x=="Najost Camp": return "나조스트캠핑장"
    elif x=="iFA캠프운악": return "캠프운악"
    elif x=="Black tree": return "블랙트리캠핑장"
    elif x=="영천 구룡산 오토&펜션": return "영천구룡산오토캠핑장"
    elif x=="(주)자연에너지-드림랜드오토캠핑장": return "드림랜드오토캠핑장"
    elif x=="소회산리멍우리협곡관광농원(캠핑장)": return "멍우리협곡캠핑장"
    elif x=="춘천Believing(빌리빙)캠핑장": return "춘천빌리빙캠핑장"
    elif x=="변산반도국립공원 고사포 야영장 (임시)": return "고사포야영장"
    elif x=="A&J오토캠핑장": return "AJ오토캠핑장"
    elif x=="Camp 1950 by 민들레울": return "camp1950"
    elif x=="(주)자연인": return "자연인글램핑"
    elif x=="(주)수동자연마을 힐링별밤수목원캠핑장": return "힐링별밤수목원캠핑장"
    elif x=="stay714": return "스테이714"
    elif x=="선녀와나무꾼": return "선녀와나무꾼캠핑장"
    elif x=="블루스카이": return "블루스카이캠핑장"
    elif x=="CLUB 596": return "클럽596"
    elif x=="비토애": return "비토애글램핑"
    elif x=="(주)연합진흥 경도 글램핑파크": return "경도글램핑파크"
    elif x=="(주)태평소금 천일염 힐링캠프": return "천일염힐링캠프"
    elif x=="누룽지": return "누룽지캠핑장"
    elif x=="포시즌": return "포시즌캠핑장"
    elif x=="마이웨이 리조트 관광": return "마이웨이리조트"

    elif "&" in x: return re.sub("&","앤", x)
    elif re.search("\(.*\)", x): return re.sub("\(.*\)","", x)
    else: return x
#     else: return x.replace(" ","").strip()
        
df["hashtag_test"] = df["name"].apply(name_for_tag)
df["hashtag_test"] = df["hashtag_test"].apply(name_for_tag)
df["hashtag"] = df["hashtag_test"].apply(lambda x: x.replace(" ","").strip())
print('modified : {count}'.format(count=len(df[df["name"]!=df['hashtag_test']])))
df[df["name"]!=df['hashtag_test']].head()

modified : 81


,name,addr,call_num,coord_x,coord_y,hashtag,hashtag_test
109,얼음골한옥펜션&캠핑장,경남 밀양시 산내면 하양지안길 11-19,,128.9528490,35.5984747,얼음골한옥펜션앤캠핑장,얼음골한옥펜션앤캠핑장
119,진부(오대산)관광농원,강원 평창군 진부면 아차골길 34-15,010-6343-6516,128.5672147,37.5555278,진부관광농원,진부관광농원
135,소백산 신선펜션 & 캠핑,충북 단양군 가곡면 새밭로 474,010-4963-7169,128.4228607,36.9958373,소백산신선펜션앤캠핑,소백산 신선펜션 앤 캠핑
177,(주)아웃오브파크,강원도 춘천시 남면 가옹개길 52-9,1522-1861-,127.5112565,37.7278127,아웃오브파크,아웃오브파크
181,505펜션&캠핑(오공오),충남 공주시 사곡면 호계황골길 43,010-7330-2505,127.0207700,36.5017433,505펜션앤캠핑,505펜션앤캠핑


In [770]:
len(df['name'].unique())

2406

In [769]:
df.to_csv("../file/campingsite.csv")

# Instagram Hashtag 수집

In [798]:
def get_postnum(df, tag_nm, num=0):
    rows_list = []
    tag_list = df[tag_nm].values.tolist()
    
    today = datetime.datetime.today() 
    today_ymd = today.strftime('%Y%m%d')
    
    cols = list(df.columns) + ['num_posts','date_time']
    df_out = pd.DataFrame(columns=cols)

    try:
        os.mkdir(new_path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
    
    chrome_path = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(chrome_path, options=options)

    for i in range(num,len(tag_list)):
        driver.get('https://www.instagram.com/explore/tags/'+tag_list[i]+'/?hl=en') 
        time.sleep(1.5) 
        source = driver.page_source 
        data = bs(source, 'html.parser') 
        tag = data.find("span",{"class": "g47SY "}) 
        try: 
            val = tag.text 
            val = int(val.replace(',' , '')) 
        except AttributeError: 
            val = 0

        info = list(df.iloc[i,:]) + [val, today]
        dict1 = dict(zip(cols,info))
        df_out = df_out.append(dict1, ignore_index=True)
        print(i,end=" ")

        if i>0 and i%100 == 0:
            print(df_out.tail(3))
            driver.quit()
            time.sleep(2)
            driver = webdriver.Chrome(chrome_path, options=options)
    driver.quit()
    return df_out


In [799]:
df_postnum = get_postnum(df, 'hashtag')

Creation of the directory ../file/ failed
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100          name                   addr       call_num      coord_x     coord_y  \
98   천동계곡 야영장   충북 단양군 단양읍 천동4길 10-8  010-2240-2458  128.4189781  36.9658204   
99     글램퍼스양평   경기 양평군 단월면 대부록길 38-3   031-772-7179  127.6821260  37.5791182   
100    깊은골캠핑장  전라북도 무주군 구천동로 1284-6    063-324-3470  127.7607566  35.9260337   

     hashtag hashtag_test num_posts                  date_time  
98   천동계곡야영장     천동계곡 야영장         0 2020-06-18 14:48:38.240288  
99    글램퍼스양평       글램퍼스양평         0 2020-06-18 14:48:38.240288  
100   깊은골캠핑장       깊은골캠핑장         0 2020-06-18 14:48:38.240288  
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121

In [802]:
df_postnum.to_csv('../file/camping_20200618.csv', index=False)

In [ ]:
df_postnum

## geocoding example

In [ ]:
cols = list(df.columns) + ['level1','level2','x','y']
df_fn = pd.DataFrame(columns=cols)
apikey = '6829FE20-AE45-312E-9D0A-418C7E77BF47'

for i in range(len(df)):
    dict0 = dict(df.iloc[i])
#     print(dict0)

    address = df['addr'][i]
    URL = 'http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={addr}&refine=true&simple=false&format=json&type=road&key={key}'.format(key=apikey, addr=address)
    r = requests.get(URL)
    data = r.json()
    try: 
        dict1 = data['response']['refined']['structure']
        dict1 = {k: dict1[k] for k in dict1.keys() & {'level1','level2'}}
        dict2 = data['response']['result']['point']
        dict0.update(dict1)
        dict0.update(dict2)
    except KeyError:
        dict1 = {'level1':None, 'level2':None, 'x':None, 'y':None}
        dict0.update(dict1)
    
    df_tmp = pd.DataFrame(json_normalize(dict0), columns=dict0.keys())
    df_fn = df_fn.append(df_tmp)
    print(df_fn.tail(5))